# MISP Feed generator

## Purpose
Allow to generate a MISP feed in real time, where each items are added on daily generated events or specfied one.

## Files
- ``generator.py`` exposes a class allowing to generate a MISP feed in real time, where each items can be added on daily generated events.
- ``fromredis.py`` uses ``generator.py`` to generate a MISP feed based on data stored in redis.
- ``server.py`` is a simple script using Flask_autoindex to serve data to MISP. 
- ``MISPItemToRedis.py`` permits to push (in redis) items to be added in MISP by the fromredis.py script.


## Installation

####  Feed generator
````
git clone https://github.com/CIRCL/PyMISP
cd examples/feed-generator-from-redis
bash install.sh # install dependencies
cp settings.default.py settings.py
vi settings.py  # adjust your settings
```

#### Activate the virtalenv
```
. ./serv-env/bin/activate
```
#### Start the feed generator
```
python3 fromredis.py
```

### Serving file to MISP
```
python3 server.py
```

## Library usage

#### Generate the feed manually
The feed generator uses the configuration provided in the file ``settings.py``.
It will create daily events in which attributes and objects will be added dynamically.

In [1]:
# default import
from feedgenerator.MISPItemToRedis import MISPItemToRedis
from feedgenerator.generator import FeedGenerator

In [2]:
# Create the FeedGenerator object
generator = FeedGenerator()

In [3]:
# Add an attribute to the daily event
attr_type = "ip-src"
attr_value = "8.8.8.8"
additional_data = {}
generator.add_attribute_to_event(attr_type, attr_value, **additional_data)

Writting event on disk                                                  
Writting event on disk                                                  
Manifest saved
Writting event on disk                                                  
Hash saved                              


True

In [4]:
# Add a cowrie object to the daily event
obj_name = "cowrie"
obj_data = { 
    "session": "session_id", 
    "username": "admin", 
    "password": "admin", 
    "protocol": "telnet"
}
generator.add_object_to_event(obj_name, **obj_data)

Writting event on disk                                                  
Hash saved                              


True

In [5]:
# Immediatly write the event to the disk (Bypassing the default flushing behavior)
generator.flush_event()

Writting event on disk                                                  


## Redis as buffer: Generate a MISP feed based on data stored in redis

### Architecture
<img src="./pics/feed-generetor-redis-arch.png" width="71%">

### Adding items to Redis

In [6]:
# create helper object
helper = MISPItemToRedis("redis_list_keyname")

In [7]:
# push an attribute to redis
helper.push_attribute("ip-src", "8.8.8.8", category="Network activity")

In [10]:
# push an object to redis
helper.push_object({ 
    "name": "cowrie", 
    "session": "session_id", 
    "username": "admin2", 
    "password": "admin2", 
    "protocol": "telnet" 
})

### Consume stored data in redis
Configuration provided in the file settings.py
```
>>> python3 fromredis.py
```
or,

In [ ]:
# Configuration provided in the file settings.py
from feedgenerator.fromredis import RedisToMISPFeed

redisToMISP = RedisToMISPFeed()
redisToMISP.consume()

### Serve data to MISP

Just run the server:
```
>>> python3 server.py
```